**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

from class_utils.download import download_file_maybe_extract
download_file_maybe_extract(DATA_HOME.format("titanic.zip"), directory="data/titanic")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Prespracovanie dát pomocou pipelines

V predchádzajúcom notebook-u sme ukázali, ako treba aplikovať na dáta predspracovanie, ak má byť celý postup opakovateľný. Daný prístup bol korektný, jeho nevýhodou však bola určitá prácnosť (a s ňou spojená možnosť omylu). Preto teraz ukážeme praktickejší spôsob, ako to isté realizovať – pomocou konceptu tzv. **pipelines**  – tiež z balíčka `scikit-learn`.

### Načítanie dát

Na úvod opäť načítajme dátovú množinu [Titanic](https://www.kaggle.com/c/titanic) a rozdeľme ju na tréningové a testovacie dáta.



In [ ]:
df = pd.read_csv("data/titanic/train.csv")
df_train, df_test = train_test_split(df, test_size=0.25,
                     stratify=df["Survived"], random_state=4)

In [ ]:
df_train.head()

Pripomeňme si aj, čo jednotlivé stĺpce obsahujú:



In [ ]:
with open("data/titanic/description", "r") as file:
    print("".join(file.readlines()))

### Selekcia stĺpcov

Ako už vieme, naša dátová množina má viacero stĺpcov. Niektoré z nich obsahujú kategorické a niektoré numerické hodnoty. Ako sme videli v predchádzajúcom notebook-u, na každý z týchto dvoch typov budeme chcieť aplikovať o trochu iný typ predspracovania.

Niektoré stĺpce pravdepodobne nebudeme chcieť použiť vôbec, pretože informácie, ktoré sú v nich obsiahnuté, buď nie sú užitočné alebo z nich užitočnú informáciu aspoň zatiaľ nevieme extrahovať. Stĺpec `PassengerId` obsahuje napríklad unikátny numerický identifikátor záznamu. Pravdepodobne nie je dobrý nápad použiť ho ako vstup modelu, pretože neobsahuje zovšeobecniteľnú informáciu – idenfikátory sú unikátne a nemali by v tomto prípade niesť žiaden informačný obsah.

Stĺpce `Name`, `Cabin` a ďalšie zovšeobecniteľné informácie môžno nesú, ak by sme ich vedeli vhodne predspracovať (súčasťou mien sú napríklad aj tituly, ktoré môžu niesť zovšeobecniteľnú informáciu; z čísla kajuty by sa zase mohlo dať zistiť, v ktorej časti lode sa nachádzala a pod.). Keďže však zatiaľ nevieme, ako také predspracovanie realizovať, tento typ stĺpcov jednoducho vynecháme.

Zvyšné stĺpce rozdelíme podľa toho, či sú numerické alebo kategorické. Stĺpec `Survived` predstavuje požadovaný výstup: ten nespracovávame spolu so vstupmi, ale osobitne (navyše, hodnoty sú 0 a 1 a už ich nie je potrebné ďalej spracovať).



In [ ]:
categorical_inputs = ["Pclass", "Sex", "Embarked"]
numeric_inputs = ["Age", "SibSp", 'Parch', 'Fare']

output = "Survived"

### Zostavenie pipeline a predspracovanie dát

Keďže numerické stĺpce chceme ošetriť iným spôsobom než kategorické, použijeme na zostavenie pipeline funciu `make_column_transformer`, ktorá nám umožňuje špecifikovať rôzne pipelines pre rôzne stĺpce. Zároveň platí, že stĺpce, ktoré vôbec neuvedieme, sa vynechajú. Predspracovanie v duchu predchádzajúceho notebook-u by sme aplikovali takto:



In [ ]:
input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OrdinalEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

Najprv pomocou funkcie `fit_transform` vytvorený pipeline objekt naladíme a zároveň aj predspracujeme pôvodnú dátovú množinu. Z dátovej množiny takisto extrahujeme stĺpec s požadovanými výstupmi. Požadované výstupy zároveň transformujeme na 1-rozmerné pole, ako to trieda `KNeighborsClassifier` očakáva.



In [ ]:
X_train = input_preproc.fit_transform(df_train[categorical_inputs+numeric_inputs])
Y_train = df_train[output].values.reshape(-1)

Na predspracovanie testovacích dát použijeme znovu ten istý pipeline objekt.

**Nezabudnime, že teraz už budeme volať len metódu `transform` a nie metódu `fit_transform`, kedže pipeline objekt už nechceme ladiť. Chceme len transformovať testovacie dáta rovnako ako sme to urobili s tréningovými.** 



In [ ]:
X_test = input_preproc.transform(df_test[categorical_inputs+numeric_inputs])
Y_test = df_test[output].values.reshape(-1)

### Tréning modelu

Ďalej nám už nezostáva nič iné, než natrénovať samotný model – môžeme použiť napríklad už známy `KNeighborsClassifier`, ktorému sme sa venovali v jednom z predchádzajúcich notebook-ov.



In [ ]:
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, Y_train)

### Testovanie modelu

Model si ďalej otestujeme na testovacích dátach.



In [ ]:
y_test = model.predict(X_test)

Zobrazíme maticu zámen a vypočítame správnosť (accuracy).



In [ ]:
cm = pd.crosstab(Y_test, y_test,
                 rownames=['actual'],
                 colnames=['predicted'])
print(cm)

In [ ]:
acc = accuracy_score(Y_test, y_test)
print("Accuracy = {}".format(acc))

### Ako sledovať, ktoré hodnoty chýbali

Keď pracujeme so stĺpcami, kde chýbajú hodnoty – či už sú numerické alebo o kategorické – je okrem doplenia chýbajúcich hodnôt často užitočné zapamätať si, ktoré hodnoty chýbali. Môže sa napríklad stať, že metóda, ktorou hodnoty dopĺňame ich systematicky nadhodnocuje alebo podhodnocuje. Ak náš model vie, ktoré hodnoty boli doplnené, môže sa naučiť také skreslenie kompenzovať.

Stĺpce s chýbajúcimi hodnotami môžeme automaticky identifikovať a aplikovať na ne transformátor `MissingIndicator`: vytvoria sa tým nové binárne stĺpce, ktoré budú indikovať, či dané hodnoty chýbali alebo nie. Prirodzene neexistuje žiadna záruka, že tento postup bude zakaždým viesť ku lepším výsledkom – môže to závisieť od dátovej množiny aj od použitej metódy.



In [ ]:
has_missing = df_train.isnull().any()
for_missing_tracking = has_missing[has_missing].keys()

In [ ]:
tracking_input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OrdinalEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs),
    
    # ---------------------
    (MissingIndicator(),
     for_missing_tracking)
    # ---------------------
)